In [1]:
import pandas as pd
import sklearn
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import manifold
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from sklearn.model_selection import cross_val_score

init_notebook_mode(connected=True)

# Jaki procent całego zbioru ma być wykorzystany do uczenia
TRAIN_DATASET_PRECENTAGE = 0.85

##
## 11 - running
## 22 - walking
## 33 - car
## 44 - bike
## 55 - upstairs
## 66 - downstairs

In [2]:
# Załadowanie zbioru cech z csv
dataset = pd.read_csv("features_fixed.csv", index_col=None)

# Stworzenie kolumn dla wektora cech. Nazwy to liczby od 1 do 25.
cols = [str(x) for x in range(1, 26)]
#Ostatnia kolumna to etykieta aktywności
cols.append("label")

dataset.columns = cols


for label in (11,22,33,44,55,66):
    print(label, dataset.loc[dataset["label"] == label].shape)

## 11 - running
## 22 - walking
## 33 - car
## 44 - bike
## 55 - upstairs
## 66 - downstairs

11 (844, 26)
22 (880, 26)
33 (1377, 26)
44 (53, 26)
55 (404, 26)
66 (220, 26)


In [3]:
# Mieszamy wczytane wiersze tak, żeby kolejność cech nie miała wpływu na trenowanie i testowanie
dataset = sklearn.utils.shuffle(dataset)

In [4]:
# Obliczenie liczby wektorów cech potrzebynch do uczenia
train_size = int(len(dataset)*TRAIN_DATASET_PRECENTAGE)

#Podział na zbiór uczący i testowy
train_dataset = dataset[:train_size]
test_dataset = dataset[train_size:]

# Oddzielamy cechy od etykiety i przypisujemy do X - zbioru danych uczących
feature_cols = dataset.columns[:-1]
X = train_dataset.loc[:, feature_cols]

# Zbiór poprawnych etykiet dla zbioru uczącego
y = train_dataset.label

In [5]:
# Skalowanie wektora cech
scaler = StandardScaler()
scaler.fit(X)

X = scaler.transform(X)

In [6]:
# Podział na X i y dla danych testowych
X_test = test_dataset.loc[:, feature_cols]
X_test = scaler.transform(X_test)
y_test = test_dataset.label

In [7]:
# Tworzymy listę klasyfikatorów które testujemy
classifiers = []

classifiers.append({"name":"SVM", "cls":SVC(C=110.0104698904004, tol=0.005103440905884713, gamma=0.06389136736841583)})
classifiers.append({"name":"MLP", "cls":MLPClassifier(solver='adam', alpha=1e-5,
                                                      hidden_layer_sizes=(200,200), 
                                                      max_iter=500, shuffle=True)})
classifiers.append({"name":"KNN", "cls":KNeighborsClassifier(n_neighbors=3)})
classifiers.append({"name":"RF", "cls":RandomForestClassifier()})

In [8]:
# Cross validate

X_cv = np.concatenate((X, X_test))
y_cv = np.concatenate((y, y_test))

cv = 2

print("Cross validation folds=%d" % (cv))

for clsf in classifiers:
    scores = cross_val_score(clsf["cls"], X_cv, y_cv, cv=cv)
    print(clsf["name"] + ": %0.2f" % (scores.mean()*100) + "%")
    print(scores)

Cross validation folds=2
SVM: 81.10%
[0.81957672 0.80243644]
MLP: 83.32%
[0.84074074 0.82574153]
KNN: 76.15%
[0.76243386 0.76059322]
RF: 76.50%
[0.76666667 0.76324153]


In [9]:
# Każdy klasyfikator uczymy i testujemy, a następnie rysujemy heatmap z wynikami
for clsf in classifiers:
    # Uczenie klasyfikatora
    clsf["cls"].fit(X, y)
    # Sprawdzenie dla danych testowych
    predictions = clsf["cls"].predict(X_test)
    # Zapis wyniku
    acc = accuracy_score(y_test, predictions)
    
    # Znormalizowana macierz
    c_matrix = confusion_matrix(y_test, predictions, labels = [11,22,33,44,55,66])
    c_matrix = c_matrix.astype('float') / c_matrix.sum(axis=1)[:, np.newaxis]
    c_matrix = c_matrix*100.0
    c_matrix = c_matrix.round(2)
    
    # Tytuły wykresów
    title = clsf["name"] + " - overall accuracy: " + str(round(acc*100,2)) + "%"
    labels = ["running", "walking", "car", "bike", "upstairs", "downstairs"]
    
    fig = ff.create_annotated_heatmap(z=c_matrix, x=labels, y=labels)
    fig.layout.update({"title": title})
    
    fig.layout.update({"yaxis":dict(autorange='reversed')})
    
    iplot(fig)


In [10]:
# # clf = SVC()
# clf = MLPClassifier(solver='lbfgs', alpha=1e-7, hidden_layer_sizes=(26, 10, 6), random_state=1, max_iter=700)
# # clf = KNeighborsClassifier(n_neighbors=7)
# # clf = SGDClassifier(loss="log")
# clf.fit(X,y)

# predictions = clf.predict(X_test)
# acc = accuracy_score(y_test, predictions)
# print("Accuracy: " + str(round(acc*100,2)) + "%")

In [11]:
# tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
# Y = tsne.fit_transform(X_test)

In [12]:
# plt.scatter(Y[:, 0], Y[:, 1], c=y_test, cmap=plt.cm.get_cmap("jet", len(list(set(y_test)))))
# plt.colorbar(ticks=range(len(list(set(y_test)))))

# plt.show()

In [13]:
# import autosklearn.classification
# automl = autosklearn.classification.AutoSklearnClassifier(
#     include_estimators=["libsvm_svc", ], exclude_estimators=None,
#     include_preprocessors=["no_preprocessing", ], exclude_preprocessors=None)

# automl.fit(X.copy(),y.copy())


In [14]:
# # automl.show_models()
# from sklearn.externals import joblib
# automl = joblib.load(".pkl")

In [15]:
# predictions = automl.predict(X_test)
# acc = accuracy_score(y_test, predictions)
# print("Accuracy: " + str(round(acc*100,2)) + "%")